In [60]:
import pandas as pd
import re
import random
import opendatasets as od

import torch
import torchtext
import torch.nn as nn
import torch.functional as F
from torch.optim import Adam

from torchtext.data import Field,LabelField,TabularDataset,BucketIterator
from torchtext.vocab import Vocab



In [61]:
device = 'cuda' if torch.cuda.is_available else 'cpu'
print(device)
print(torch.__version__)
print(torchtext.__version__)

cuda
2.1.0+cu118
0.6.0


In [62]:
od.download('https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset')

Skipping, found downloaded files in ".\fake-and-real-news-dataset" (use force=True to force download)


### Importing Dataset

In [63]:
real_df = pd.read_csv('./fake-and-real-news-dataset/True.csv')
fake_df = pd.read_csv('./fake-and-real-news-dataset/Fake.csv')

In [64]:
real_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [65]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


### Preprocessing and Merging

In [66]:
real_df.drop(['date','subject'],axis=1,inplace=True)
fake_df.drop(['date','subject'],axis=1,inplace=True)

In [67]:
real_df['Label'] = 1.0
fake_df['Label'] = 0.0

In [68]:
real_df['TEXT'] = real_df['title']+real_df['text']
real_df.drop(['title','text'],axis=1,inplace=True)

In [69]:
fake_df['TEXT'] = fake_df['title']+fake_df['text']
fake_df.drop(['title','text'],axis=1,inplace=True)

In [70]:
real_df.head(3)

,Label,TEXT
0,1.0,"As U.S. budget fight looms, Republicans flip t..."
1,1.0,U.S. military to accept transgender recruits o...
2,1.0,Senior U.S. Republican senator: 'Let Mr. Muell...


In [71]:
fake_df.head(3)

,Label,TEXT
0,0.0,Donald Trump Sends Out Embarrassing New Year’...
1,0.0,Drunk Bragging Trump Staffer Started Russian ...
2,0.0,Sheriff David Clarke Becomes An Internet Joke...


In [72]:
news_df = pd.concat([fake_df,real_df],ignore_index=True)
news_df

,Label,TEXT
0,0.0,Donald Trump Sends Out Embarrassing New Year’...
1,0.0,Drunk Bragging Trump Staffer Started Russian ...
2,0.0,Sheriff David Clarke Becomes An Internet Joke...
3,0.0,Trump Is So Obsessed He Even Has Obama’s Name...
4,0.0,Pope Francis Just Called Out Donald Trump Dur...
...,...,...
44893,1.0,'Fully committed' NATO backs new U.S. approach...
44894,1.0,LexisNexis withdrew two products from Chinese ...
44895,1.0,Minsk cultural hub becomes haven from authorit...
44896,1.0,Vatican upbeat on possibility of Pope Francis ...


In [73]:
news_df.rename(columns={'Label':'LABEL'},inplace=True)

In [74]:
#news_df.to_csv('news_data.csv')

### NLP Tasks

In [75]:
news_df['TEXT'] = news_df['TEXT'].apply(lambda x:x.lower())
news_df['TEXT'] = news_df['TEXT'].apply(lambda x:re.sub('[^a-zA-z0-9\s]','',x))

In [76]:
TEXT = Field(tokenize='spacy',tokenizer_language='en_core_web_sm')
LABEL = LabelField(dtype=torch.float32)
Fields =[(None, None),('LABEL',LABEL),('TEXT',TEXT)]

In [77]:
news_ds = TabularDataset(
    path='news_data.csv', format='csv',
    skip_header=True, fields=Fields)

In [78]:
train_data, test_data = news_ds.split(
    split_ratio=[0.8, 0.2],
    random_state=random.seed(47))

print(f'Num Train: {len(train_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 35918
Num Test: 8980


In [79]:
train_data, valid_data = train_data.split(
    split_ratio=[0.85, 0.15],
    random_state=random.seed(47))

print(f'Num Train: {len(train_data)}')
print(f'Num Validation: {len(valid_data)}')

Num Train: 30530
Num Validation: 5388


In [80]:
print(vars(train_data.examples[0]))

{'LABEL': '0', 'TEXT': ['sickening', 'reason', 'cophating', 'racist', '49ers', 'qb', 'colin', 'kaepernick', 'just', 'announced', 'hell', 'now', 'stand', 'for', 'national', 'anthemit', 'looks', 'like', 'veteran', 'san', 'francisco', '49ers', 'quarterback', 'colin', 'kaepernick', 'might', 'be', 'done', 'with', 'his', 'national', 'anthem', 'protests', '  ', 'now', 'that', 'he', 's', 'looking', 'for', 'a', 'new', 'nfl', 'contractthe', 'controversial', 'player', 'will', 'stand', 'for', ' ', 'the', 'starspangled', 'banner', ' ', 'next', 'season', 'according', 'to', 'sources', 'who', 'spoke', 'to', 'espn', 'thursday', 'kaepernick', 'first', 'made', 'headlines', 'in', 'august', '2016', 'for', 'refusing', 'to', 'stand', 'during', 'his', 'team', 's', 'preseason', 'game', 'against', 'the', 'green', 'bay', 'packers', 'citing', 'the', ' ', 'oppression', ' ', 'of', 'black', 'people', 'in', 'the', 'united', 'stateskaepernick', 'explained', 'his', 'refusal', 'to', 'stand', 'for', 'the', 'starspangled'

In [81]:
TEXT.build_vocab(train_data,max_size=25000)
LABEL.build_vocab(train_data)

print("Vocabualry Size is %d" % len(TEXT.vocab))

Vocabualry Size is 25002


In [82]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 687383), ('to', 373633), ('of', 304821), ('a', 279350), ('and', 278517), (' ', 253551), ('in', 240090), ('that', 162418), ('on', 133236), ('for', 121885), ('s', 121271), ('is', 114708), ('he', 91309), ('said', 88706), ('trump', 86253), ('it', 84920), ('with', 81824), ('was', 79164), ('as', 70735), ('his', 66202)]


In [83]:
train_dl,valid_dl,test_dl = BucketIterator.splits((train_data,valid_data,test_data),
                                           batch_size=32,
                                           sort_within_batch=False,
                                           sort_key=lambda x: len(x.TEXT),device=device)

### RNN-LSTM Model Building

In [84]:
class RNN_LSTM(nn.Module):
    '''
    Building an RNN_LSTM model 
    '''
    def __init__(self,input_dim,embedding_dim,hidden_dim,output_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim,hidden_dim)
        self.fc = nn.Linear(hidden_dim,output_dim)
    
    def forward(self,x):
        x = self.embedding(x)
        out,(hidden,cell) = self.lstm(x)
        hidden = hidden.squeeze_(0)
        logits = self.fc(hidden)
        out = torch.nn.functional.softmax(logits,dim=-1)
        return out

In [85]:
model = RNN_LSTM(input_dim=len(TEXT.vocab),
                 embedding_dim=128,
                 hidden_dim=256,
                 output_dim=2)


model = model.to(device)


In [86]:
# Embedding matrix shape
it = iter(model.parameters())
next(it)[0].shape

torch.Size([128])

### Training and Evaluation

In [87]:
torch.cuda.get_device_properties(0).name

'NVIDIA GeForce RTX 4070 Laptop GPU'

In [88]:
num_epochs = 10
optimizer = Adam(model.parameters(),lr=3e-4)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
for epoch in range(num_epochs):
    train_acc = 0.0
    train_loss = 0.0

    for i, batch_data in enumerate(train_dl):
        text = batch_data.TEXT.to(device)
        labels = batch_data.LABEL.type(torch.LongTensor)
        labels = labels.to(device)   
        
        #forward
        preds = model(text)
        preds = preds.to(torch.float32)
        #loss
        loss = loss_fn(preds,labels)
        #backprop
        optimizer.zero_grad()
        loss.backward()

        #update wts
        optimizer.step()

        train_loss += loss.detach().item()
        train_acc += (torch.argmax(preds,1).flatten() == labels).type(torch.float32).mean().item()

    print("Epoch: %d || Loss: %.2f || Train Acc: %.2f" %(epoch,train_loss/i ,train_acc/i))


In [94]:
model.parameters()

<generator object Module.parameters at 0x000001C7BC466650>